## Web Scrapping de Estados Financieros

Se presenta un código a partir del cual se puede obtener los estados financieros de los principales emisores del mercado principal, obtenidos de la página de la Superintendencia del Mercado de Valores (SMV)


In [1]:
# Importación de librería
# ===============================

import openpyxl
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
import time
import os
import shutil

In [2]:
# Configuración de Webbrowser
# ===============================

options = webdriver.ChromeOptions()
options.add_argument('--headless')                            # Para ejecutar el browser sin una interfaz de usuario
options.add_argument('--no-sandbox')                          # Necesario cuando no se tiene configurado a un usuario
options.add_argument('--disable-dev-shm-usage')               # Para incrementar el tamaño de memoria en un container
browser = webdriver.Chrome('chromedriver', options=options)   # Inicilizamos el webdriver haciendo referencia al chromedriver (path)

In [3]:
# Llamando página web de la información financiera de los emisores del mercado principal
# ===========================================================================

browser.get('https://www.smv.gob.pe/SIMV/Frm_InformacionFinanciera?data=A70181B60967D74090DCD93C4920AA1D769614EC12')

time.sleep(5)

# Seleccionando tipo, período y año:
# ===========================================================================

### Tipo: Consolidado
tipo = '//*[@id="MainContent_cboTipo_1"]'
tipo_radio = browser.find_element(By.XPATH, tipo)
browser.execute_script('arguments[0].click()', tipo_radio)

### Periodo: Anual
periodo = '//*[@id="MainContent_cboPeriodo_1"]'
periodo_radio = browser.find_element(By.XPATH, periodo)
browser.execute_script('arguments[0].click()', periodo_radio)

time.sleep(15)

## Años a utilizar
años = browser.find_elements(By.XPATH, '//*[@id="MainContent_cboAnio"]/option')
years=list()
for año in años[3:len(range(2004,2021))+4]: # 2021 - 2004
    time.sleep(5)
    año_smv=año.text
    years.append(año_smv)            

In [4]:
# Iteración por años:
# ===============================
for year in years:
    time.sleep(5)
    print(year)
    xpath_select = '//*[@id="MainContent_cboAnio"]'
    Select(browser.find_element(By.XPATH, xpath_select)).select_by_visible_text(year)

    ## Dando click a buscar:
    buscar = '//*[@id="MainContent_cbBuscar"]'
    browser.find_element(By.XPATH, buscar).click()


    # Iteración por páginas:
    # ===============================
    time.sleep(30)
    xpath_siguiente = '//*[@id="MainContent_grdInfoFinanciera"]/tbody/tr[52]/td/table/tbody/tr/td'
    e_siguiente = browser.find_elements(By.XPATH, xpath_siguiente)
    idx=0

    for number in range(1,3): ##range(1,len(e_siguiente)+1) --> para todas las hojas
        xpath_siguiente = '//*[@id="MainContent_grdInfoFinanciera"]//td/table/tbody/tr/td'
        e_siguiente = browser.find_elements(By.XPATH, xpath_siguiente)
        e_siguiente[idx].click()
        idx += 1
        print(f'Recolectando datos de la pagina {idx}')

        # Iteración por links de los estados financieros de las empresas:
        # ===============================
        time.sleep(10)
        links_ef = browser.find_elements(By.XPATH, '//*[@id="MainContent_grdInfoFinanciera"]/tbody/tr/td/a[contains(@title,"Estados Financieros")]')
        for link in links_ef[0:2]:
            link_ef=link.get_attribute('href')

            ## Definiendo nueva ventana
            browser.execute_script('window.open();')
            window_before = browser.window_handles[0]
            window_after = browser.window_handles[1]
            
            ## Cambiando de ventana
            browser.switch_to.window(window_after)
            browser.get(link_ef)

            ## Obteniendo los nombres de la empresas:
            xpath_top = '//*[@id="lblEmpresa"]'
            nombre_emp = browser.find_elements(By.XPATH, xpath_top)
            emp = nombre_emp[0].text

            ## Descargando el excel
            time.sleep(5)
            excel = '//*[@id="cbExcel"]'
            browser.find_element(By.XPATH, excel).click()
            
            ## Obteniendo la ruta de descarga del archivo
            time.sleep(5)
            download_folder = 'C:/Users/Usuario/Downloads/'
            file_name = max([download_folder + "/" + f for f in os.listdir(download_folder)], key=os.path.getctime)

            new_file_name = f"{year}_{emp}"
            new_file_location = 'D:/SMV/'

            ## Verificando si el archivo ya existe en la carpeta de destino (A veces presentan el estado financiero en dos fechas)
            i = 1
            while os.path.isfile(new_file_location + new_file_name + '.' + file_name.split('.')[-1]):
                 new_file_name = f"{year}_{emp}_{i}"
                 i += 1

            ## Copiando el archivo a la carpeta de destino con el nuevo nombre

            shutil.copy(file_name, new_file_location + new_file_name + '.' + file_name.split('.')[-1])

            ## Eliminando el archivo original
            os.remove(file_name)

            browser.close()
            browser.switch_to.window(window_before)

    time.sleep(10)
    xpath_siguiente = '//*[@id="MainContent_grdInfoFinanciera"]//td/table/tbody/tr/td'
    e_siguiente = browser.find_elements(By.XPATH, xpath_siguiente)
    e_siguiente[0].click()

browser.close()
browser.quit()

2021
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2020
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2019
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2018
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2017
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2016
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2015
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2014
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2013
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2012
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2011
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2010
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2009
Recolectando datos de la pagina 1
Recolectando datos de la pagina 2
2008
Recolectando datos de la pagina 1
Recolectando